## Objective of Task D


A. Build a baseline model (5 marks) 

Use the Keras library to build a neural network with the following:

- One hidden layer of 10 nodes, and a ReLU activation function

- Use the adam optimizer and the mean squared error  as the loss function.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing. You can use the train_test_splithelper function from Scikit-learn.

2. Train the model on the training data using 50 epochs.

3. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

5. Report the mean and the standard deviation of the mean squared errors.

B. Repeat Part A but use a normalized version of the data. Recall that one way to normalize the data is by subtracting the mean from the individual predictors and dividing by the standard deviation.

D. Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.


<a id="item31"></a>


## Download and Clean Dataset


Let's start by importing the <em>pandas</em> and the Numpy libraries.


In [1]:
import pandas as pd
import numpy as np

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>


Let's download the data and read it into a <em>pandas</em> dataframe.


In [2]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 


#### Let's check how many data points we have.


In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.


Let's check the dataset for any missing values.


In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.


The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.


In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


The task B step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


<a id="item2"></a>


In [8]:
predictors = predictors_norm

In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

## Import Keras


#### Let's go ahead and import the Keras library


In [10]:
import keras

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.


In [11]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import ReLU
from keras import activations
from keras import layers

<a id='item33'></a>


## Build a Neural Network


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [12]:
n_cols = predictors.shape[1]
n_cols

8

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, input_shape=(n_cols,)))
    model.add(layers.Activation(activations.relu))
    model.add(Dense(10, input_shape=(n_cols,)))
    model.add(layers.Activation(activations.relu))
    model.add(Dense(10, input_shape=(n_cols,)))
    model.add(layers.Activation(activations.relu))

    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Bulid pipeline for 50 random splits and models

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [15]:
predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size = 0.3)

Let's call the function now to create our model.


In [16]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the _fit_ method. We will leave out 30% of the data for validation and we will train the model.


In [17]:
# fit the model
MSE_list = list()
for iteration in range(50):
  predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, test_size = 0.3)
  model.fit(predictors_train, target_train, validation_split=0.3, epochs=50, verbose=2)
  MSE = mean_squared_error(target_test, model.predict(predictors_test))
  MSE_list.append(MSE)

Epoch 1/50
16/16 - 1s - loss: 1613.3921 - val_loss: 1515.0029
Epoch 2/50
16/16 - 0s - loss: 1595.1090 - val_loss: 1497.5452
Epoch 3/50
16/16 - 0s - loss: 1574.9730 - val_loss: 1477.0769
Epoch 4/50
16/16 - 0s - loss: 1549.8417 - val_loss: 1450.8384
Epoch 5/50
16/16 - 0s - loss: 1518.2477 - val_loss: 1417.0326
Epoch 6/50
16/16 - 0s - loss: 1476.3981 - val_loss: 1374.0242
Epoch 7/50
16/16 - 0s - loss: 1423.5133 - val_loss: 1316.3595
Epoch 8/50
16/16 - 0s - loss: 1353.2711 - val_loss: 1240.3733
Epoch 9/50
16/16 - 0s - loss: 1260.7935 - val_loss: 1142.1097
Epoch 10/50
16/16 - 0s - loss: 1143.6227 - val_loss: 1017.3951
Epoch 11/50
16/16 - 0s - loss: 1001.2915 - val_loss: 870.8998
Epoch 12/50
16/16 - 0s - loss: 838.0506 - val_loss: 711.6470
Epoch 13/50
16/16 - 0s - loss: 669.3358 - val_loss: 553.5855
Epoch 14/50
16/16 - 0s - loss: 510.2469 - val_loss: 422.6797
Epoch 15/50
16/16 - 0s - loss: 391.0295 - val_loss: 326.7961
Epoch 16/50
16/16 - 0s - loss: 311.7848 - val_loss: 272.7996
Epoch 17/50


Epoch 36/50
16/16 - 0s - loss: 87.0553 - val_loss: 114.2697
Epoch 37/50
16/16 - 0s - loss: 86.1401 - val_loss: 113.3943
Epoch 38/50
16/16 - 0s - loss: 85.9286 - val_loss: 112.9631
Epoch 39/50
16/16 - 0s - loss: 85.3680 - val_loss: 112.6249
Epoch 40/50
16/16 - 0s - loss: 84.6786 - val_loss: 111.8074
Epoch 41/50
16/16 - 0s - loss: 84.7057 - val_loss: 111.4023
Epoch 42/50
16/16 - 0s - loss: 84.1179 - val_loss: 110.5043
Epoch 43/50
16/16 - 0s - loss: 83.4049 - val_loss: 110.1299
Epoch 44/50
16/16 - 0s - loss: 82.9841 - val_loss: 108.8376
Epoch 45/50
16/16 - 0s - loss: 82.5413 - val_loss: 108.1908
Epoch 46/50
16/16 - 0s - loss: 81.9427 - val_loss: 107.8167
Epoch 47/50
16/16 - 0s - loss: 81.5857 - val_loss: 106.5639
Epoch 48/50
16/16 - 0s - loss: 80.7097 - val_loss: 105.8074
Epoch 49/50
16/16 - 0s - loss: 80.3243 - val_loss: 104.5952
Epoch 50/50
16/16 - 0s - loss: 80.1763 - val_loss: 104.0279
Epoch 1/50
16/16 - 0s - loss: 91.1963 - val_loss: 80.3912
Epoch 2/50
16/16 - 0s - loss: 89.0857 - va

16/16 - 0s - loss: 46.8867 - val_loss: 49.1711
Epoch 26/50
16/16 - 0s - loss: 46.7788 - val_loss: 49.5983
Epoch 27/50
16/16 - 0s - loss: 46.7087 - val_loss: 49.3273
Epoch 28/50
16/16 - 0s - loss: 47.1703 - val_loss: 49.6493
Epoch 29/50
16/16 - 0s - loss: 46.2393 - val_loss: 49.8334
Epoch 30/50
16/16 - 0s - loss: 46.3350 - val_loss: 49.3749
Epoch 31/50
16/16 - 0s - loss: 46.2761 - val_loss: 49.5029
Epoch 32/50
16/16 - 0s - loss: 46.3006 - val_loss: 49.2589
Epoch 33/50
16/16 - 0s - loss: 45.8560 - val_loss: 49.1261
Epoch 34/50
16/16 - 0s - loss: 46.1123 - val_loss: 49.2402
Epoch 35/50
16/16 - 0s - loss: 45.8644 - val_loss: 49.1750
Epoch 36/50
16/16 - 0s - loss: 45.5680 - val_loss: 48.7503
Epoch 37/50
16/16 - 0s - loss: 45.6786 - val_loss: 48.8937
Epoch 38/50
16/16 - 0s - loss: 45.5678 - val_loss: 48.5897
Epoch 39/50
16/16 - 0s - loss: 45.3425 - val_loss: 48.8972
Epoch 40/50
16/16 - 0s - loss: 45.5227 - val_loss: 48.8661
Epoch 41/50
16/16 - 0s - loss: 45.2971 - val_loss: 48.7385
Epoch 42/

Epoch 15/50
16/16 - 0s - loss: 37.3423 - val_loss: 47.8681
Epoch 16/50
16/16 - 0s - loss: 36.9008 - val_loss: 48.0981
Epoch 17/50
16/16 - 0s - loss: 36.5450 - val_loss: 48.0419
Epoch 18/50
16/16 - 0s - loss: 36.3382 - val_loss: 48.0755
Epoch 19/50
16/16 - 0s - loss: 36.5515 - val_loss: 48.3774
Epoch 20/50
16/16 - 0s - loss: 36.3599 - val_loss: 48.3215
Epoch 21/50
16/16 - 0s - loss: 36.2689 - val_loss: 47.7823
Epoch 22/50
16/16 - 0s - loss: 36.3674 - val_loss: 48.2656
Epoch 23/50
16/16 - 0s - loss: 36.5138 - val_loss: 48.1478
Epoch 24/50
16/16 - 0s - loss: 36.3247 - val_loss: 47.8439
Epoch 25/50
16/16 - 0s - loss: 36.1959 - val_loss: 48.5193
Epoch 26/50
16/16 - 0s - loss: 36.1393 - val_loss: 48.1633
Epoch 27/50
16/16 - 0s - loss: 35.7964 - val_loss: 48.6661
Epoch 28/50
16/16 - 0s - loss: 35.8915 - val_loss: 48.4368
Epoch 29/50
16/16 - 0s - loss: 36.2624 - val_loss: 47.8794
Epoch 30/50
16/16 - 0s - loss: 35.8832 - val_loss: 48.1388
Epoch 31/50
16/16 - 0s - loss: 35.6917 - val_loss: 48.20

Epoch 5/50
16/16 - 0s - loss: 32.9332 - val_loss: 42.9883
Epoch 6/50
16/16 - 0s - loss: 32.6637 - val_loss: 43.4534
Epoch 7/50
16/16 - 0s - loss: 32.5984 - val_loss: 43.3938
Epoch 8/50
16/16 - 0s - loss: 32.5303 - val_loss: 43.1575
Epoch 9/50
16/16 - 0s - loss: 32.6441 - val_loss: 43.3604
Epoch 10/50
16/16 - 0s - loss: 32.6366 - val_loss: 43.7353
Epoch 11/50
16/16 - 0s - loss: 32.5898 - val_loss: 43.3171
Epoch 12/50
16/16 - 0s - loss: 32.5154 - val_loss: 43.7524
Epoch 13/50
16/16 - 0s - loss: 32.1445 - val_loss: 42.7244
Epoch 14/50
16/16 - 0s - loss: 32.1122 - val_loss: 43.3700
Epoch 15/50
16/16 - 0s - loss: 32.1139 - val_loss: 43.5562
Epoch 16/50
16/16 - 0s - loss: 32.2025 - val_loss: 43.5379
Epoch 17/50
16/16 - 0s - loss: 31.9624 - val_loss: 44.3708
Epoch 18/50
16/16 - 0s - loss: 31.7896 - val_loss: 43.4851
Epoch 19/50
16/16 - 0s - loss: 32.1332 - val_loss: 43.1838
Epoch 20/50
16/16 - 0s - loss: 32.0371 - val_loss: 44.9147
Epoch 21/50
16/16 - 0s - loss: 31.9450 - val_loss: 43.1955
Ep

...

Epoch 15/50
16/16 - 0s - loss: 29.8182 - val_loss: 27.6144
Epoch 16/50
16/16 - 0s - loss: 29.3200 - val_loss: 27.4193
Epoch 17/50
16/16 - 0s - loss: 29.4572 - val_loss: 27.3044
Epoch 18/50
16/16 - 0s - loss: 30.1616 - val_loss: 27.6134
Epoch 19/50
16/16 - 0s - loss: 29.4853 - val_loss: 27.7665
Epoch 20/50
16/16 - 0s - loss: 29.2278 - val_loss: 27.5181
Epoch 21/50
16/16 - 0s - loss: 29.3154 - val_loss: 28.1957
Epoch 22/50
16/16 - 0s - loss: 29.8727 - val_loss: 27.8337
Epoch 23/50
16/16 - 0s - loss: 29.2169 - val_loss: 28.1511
Epoch 24/50
16/16 - 0s - loss: 29.1455 - val_loss: 27.9825
Epoch 25/50
16/16 - 0s - loss: 29.2352 - val_loss: 28.5790
Epoch 26/50
16/16 - 0s - loss: 29.5047 - val_loss: 27.9265
Epoch 27/50
16/16 - 0s - loss: 29.0111 - val_loss: 28.6477
Epoch 28/50
16/16 - 0s - loss: 28.9819 - val_loss: 28.4501
Epoch 29/50
16/16 - 0s - loss: 28.9005 - val_loss: 28.7977
Epoch 30/50
16/16 - 0s - loss: 29.1793 - val_loss: 28.3910
Epoch 31/50
16/16 - 0s - loss: 29.0654 - val_loss: 28.67

Epoch 5/50
16/16 - 0s - loss: 25.8589 - val_loss: 34.7456
Epoch 6/50
16/16 - 0s - loss: 25.3235 - val_loss: 36.4392
Epoch 7/50
16/16 - 0s - loss: 25.2160 - val_loss: 35.4426
Epoch 8/50
16/16 - 0s - loss: 24.9991 - val_loss: 35.9729
Epoch 9/50
16/16 - 0s - loss: 25.8886 - val_loss: 37.1456
Epoch 10/50
16/16 - 0s - loss: 25.2271 - val_loss: 35.5781
Epoch 11/50
16/16 - 0s - loss: 25.3212 - val_loss: 36.6204
Epoch 12/50
16/16 - 0s - loss: 24.8035 - val_loss: 35.1661
Epoch 13/50
16/16 - 0s - loss: 24.3565 - val_loss: 36.7478
Epoch 14/50
16/16 - 0s - loss: 24.7409 - val_loss: 35.8159
Epoch 15/50
16/16 - 0s - loss: 24.3704 - val_loss: 35.9355
Epoch 16/50
16/16 - 0s - loss: 24.6363 - val_loss: 36.0492
Epoch 17/50
16/16 - 0s - loss: 24.3271 - val_loss: 36.1164
Epoch 18/50
16/16 - 0s - loss: 24.0809 - val_loss: 35.9461
Epoch 19/50
16/16 - 0s - loss: 24.2970 - val_loss: 36.0340
Epoch 20/50
16/16 - 0s - loss: 24.2699 - val_loss: 36.2977
Epoch 21/50
16/16 - 0s - loss: 24.4818 - val_loss: 36.1092
Ep

Epoch 45/50
16/16 - 0s - loss: 27.4086 - val_loss: 29.3777
Epoch 46/50
16/16 - 0s - loss: 27.3175 - val_loss: 29.9457
Epoch 47/50
16/16 - 0s - loss: 27.2357 - val_loss: 29.8175
Epoch 48/50
16/16 - 0s - loss: 27.7037 - val_loss: 29.6186
Epoch 49/50
16/16 - 0s - loss: 27.2678 - val_loss: 29.6942
Epoch 50/50
16/16 - 0s - loss: 27.0670 - val_loss: 29.8033
Epoch 1/50
16/16 - 0s - loss: 31.5517 - val_loss: 26.7503
Epoch 2/50
16/16 - 0s - loss: 31.2589 - val_loss: 27.1232
Epoch 3/50
16/16 - 0s - loss: 30.9914 - val_loss: 26.7631
Epoch 4/50
16/16 - 0s - loss: 30.7288 - val_loss: 27.0999
Epoch 5/50
16/16 - 0s - loss: 29.9134 - val_loss: 26.7433
Epoch 6/50
16/16 - 0s - loss: 29.7531 - val_loss: 27.1777
Epoch 7/50
16/16 - 0s - loss: 29.7013 - val_loss: 26.6372
Epoch 8/50
16/16 - 0s - loss: 30.1759 - val_loss: 27.7753
Epoch 9/50
16/16 - 0s - loss: 29.9042 - val_loss: 26.7970
Epoch 10/50
16/16 - 0s - loss: 29.4146 - val_loss: 27.1826
Epoch 11/50
16/16 - 0s - loss: 28.9295 - val_loss: 26.8986
Epoch 

Epoch 35/50
16/16 - 0s - loss: 25.5113 - val_loss: 26.5957
Epoch 36/50
16/16 - 0s - loss: 25.4574 - val_loss: 27.2800
Epoch 37/50
16/16 - 0s - loss: 25.2504 - val_loss: 26.9017
Epoch 38/50
16/16 - 0s - loss: 25.2684 - val_loss: 26.6673
Epoch 39/50
16/16 - 0s - loss: 25.2729 - val_loss: 26.8713
Epoch 40/50
16/16 - 0s - loss: 25.3536 - val_loss: 27.1161
Epoch 41/50
16/16 - 0s - loss: 25.2423 - val_loss: 26.4238
Epoch 42/50
16/16 - 0s - loss: 25.0696 - val_loss: 27.3493
Epoch 43/50
16/16 - 0s - loss: 25.6890 - val_loss: 27.3228
Epoch 44/50
16/16 - 0s - loss: 25.1070 - val_loss: 27.2233
Epoch 45/50
16/16 - 0s - loss: 25.0703 - val_loss: 26.6203
Epoch 46/50
16/16 - 0s - loss: 25.0781 - val_loss: 27.5345
Epoch 47/50
16/16 - 0s - loss: 25.1473 - val_loss: 26.7761
Epoch 48/50
16/16 - 0s - loss: 25.0769 - val_loss: 27.0410
Epoch 49/50
16/16 - 0s - loss: 25.3289 - val_loss: 26.8339
Epoch 50/50
16/16 - 0s - loss: 25.0414 - val_loss: 27.2272
Epoch 1/50
16/16 - 0s - loss: 26.3222 - val_loss: 25.029

Epoch 25/50
16/16 - 0s - loss: 25.0438 - val_loss: 30.9225
Epoch 26/50
16/16 - 0s - loss: 24.9040 - val_loss: 32.0726
Epoch 27/50
16/16 - 0s - loss: 24.9681 - val_loss: 31.3736
Epoch 28/50
16/16 - 0s - loss: 24.7174 - val_loss: 31.1141
Epoch 29/50
16/16 - 0s - loss: 24.9134 - val_loss: 31.6392
Epoch 30/50
16/16 - 0s - loss: 24.7127 - val_loss: 30.8753
Epoch 31/50
16/16 - 0s - loss: 24.8506 - val_loss: 32.1667
Epoch 32/50
16/16 - 0s - loss: 24.9445 - val_loss: 30.9052
Epoch 33/50
16/16 - 0s - loss: 24.9674 - val_loss: 31.7097
Epoch 34/50
16/16 - 0s - loss: 24.6610 - val_loss: 30.8063
Epoch 35/50
16/16 - 0s - loss: 24.8901 - val_loss: 31.3513
Epoch 36/50
16/16 - 0s - loss: 24.6271 - val_loss: 30.8643
Epoch 37/50
16/16 - 0s - loss: 25.0356 - val_loss: 31.4178
Epoch 38/50
16/16 - 0s - loss: 24.8196 - val_loss: 31.9396
Epoch 39/50
16/16 - 0s - loss: 24.7196 - val_loss: 30.7494
Epoch 40/50
16/16 - 0s - loss: 24.9443 - val_loss: 32.4298
Epoch 41/50
16/16 - 0s - loss: 25.6325 - val_loss: 31.24

In [18]:
np.mean(MSE_list)

41.45160450697827

In [19]:
np.std(MSE_list)

21.386751951005003

How does the mean of the mean squared errors compare to that from Step A?

<br>Task B MSE Mean: 61.62034454414778
<br>Task B MSE Standard Deviation: 89.83764137484638
<br>Task C MSE Mean: 38.08983255433161
<br>Task C MSE Standard Deviation: 29.455900356073318

Adding more complcaed (deeper) network can provide a better model (more complex solutions used to converge)